In [1]:
import arviz as az
import pyro
import torch
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS, Predictive
import pandas as pd
from torch.autograd import Variable
import numpy as np
import xarray
xarray.set_options(display_style="html");

pyro.enable_validation(True)
pyro.set_rng_seed(0)

In [2]:
#read data
data = pd.read_csv("linear_regression_data.csv", index_col=0)
time = torch.tensor(data.time.values)
slack_comments = torch.tensor(data.comments.values)
github_commits = torch.tensor(data.commits.values)
names = data.index.values
N = torch.tensor(len(names))
data

,comments,commits,time
Alice,7500,25,4.5
Bob,10100,32,6.0
Cole,18600,49,7.0
Danielle,25200,66,12.0
Erika,27500,96,18.0


In [3]:
# data for out of sample predictions
candidate_devs = ["Francis", "Gerard"]
candidate_devs_time = torch.tensor([3.6, 5.1])

In [4]:
data_dict = {
    "developer": N,
    "time_since_joined": time
}

predictions_dict = {
    "developer": torch.tensor(2),
    "time_since_joined": candidate_devs_time
}

dims={
    "slack_comments": ["developer"],
    "github_commits": ["developer"],
    "time_since_joined": ["developer"],
}

pred_dims = {
    "slack_comments": ["developer_pred"],
    "github_commits": ["developer_pred"],
    "time_since_joined": ["developer_pred"]
}

coords = {
    "developer": names,
    "developer_pred": candidate_devs
}

def model(developer, time_since_joined, slack_comments=None, github_commits=None):
    b_sigma = abs(pyro.sample('b_sigma', dist.Normal(0, 300)))
    c_sigma = abs(pyro.sample('c_sigma', dist.Normal(0, 6)))
    b0 = pyro.sample("b0", dist.Normal(0, 200))
    b1 = pyro.sample("b1", dist.Normal(0, 200))
    c0 = pyro.sample("c0", dist.Normal(0, 10))
    c1 = pyro.sample("c1", dist.Normal(0, 10))

    with pyro.plate('developer', developer):
        slack = pyro.sample("slack_comments", dist.Normal(b0 + b1 * time_since_joined, b_sigma), obs=slack_comments)
        github = pyro.sample("github_commits", dist.Normal(c0 + c1 * time_since_joined, c_sigma), obs=github_commits)
        return slack, github


nuts_kernel = NUTS(model, jit_compile=True, ignore_jit_warnings=True)
mcmc = MCMC(nuts_kernel, num_samples=50, warmup_steps=50,
            num_chains=3, disable_progbar=True)
mcmc.run(**data_dict, slack_comments=slack_comments, github_commits=github_commits)
posterior_samples = mcmc.get_samples()
posterior_predictive = Predictive(model, posterior_samples).forward(**data_dict)
prior = Predictive(model, num_samples=50).forward(**data_dict)
predictions = Predictive(model, posterior_samples).forward(**predictions_dict)

pyro_data = az.from_pyro(mcmc,
                         prior=prior,
                         posterior_predictive=posterior_predictive,
                         predictions=predictions,
                         constant_data={"time_since_joined": time},
                         predictions_constant_data={"time_since_joined": candidate_devs_time},
                         coords=coords,
                         dims=dims,
                         pred_dims=pred_dims)

In [5]:
pyro_data

Inference data with groups:
	> posterior
	> sample_stats
	> log_likelihood
	> posterior_predictive
	> observed_data
	> constant_data
	> prior
	> prior_predictive
	> predictions
	> predictions_constant_data

In [6]:
pyro_data.posterior

<xarray.Dataset>
Dimensions:  (chain: 3, draw: 50)
Coordinates:
  * chain    (chain) int64 0 1 2
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 9 ... 40 41 42 43 44 45 46 47 48 49
Data variables:
    b_sigma  (chain, draw) float32 -2356.9644 -2356.9644 ... -1674.0819
    c_sigma  (chain, draw) float32 7.4192386 7.4192386 ... 4.534351 4.534351
    b0       (chain, draw) float32 21.524757 21.524757 ... 7.4334 7.4334
    b1       (chain, draw) float32 106.720245 106.720245 ... 1651.8323 1651.8323
    c0       (chain, draw) float32 5.440959 5.440959 ... 2.5925744 2.5925744
    c1       (chain, draw) float32 6.21757 6.21757 6.21757 ... 5.207806 5.207806
Attributes:
    created_at:                 2020-02-28T13:03:09.031403
    arviz_version:              0.6.1
    inference_library:          pyro
    inference_library_version:  1.2.1

In [7]:
pyro_data.prior

<xarray.Dataset>
Dimensions:  (chain: 1, draw: 50)
Coordinates:
  * chain    (chain) int64 0
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 9 ... 40 41 42 43 44 45 46 47 48 49
Data variables:
    b_sigma  (chain, draw) float32 -489.7559 48.63051 ... 104.658104 -140.77742
    c_sigma  (chain, draw) float32 -1.7355318 1.2556456 ... 6.1227303 1.2456974
    b0       (chain, draw) float32 -0.1493915 -115.61681 ... 327.99673 15.553215
    b1       (chain, draw) float32 138.74286 -314.8093 ... -451.88785 -29.347027
    c0       (chain, draw) float32 17.704554 3.2082124 ... 5.1072736 -16.745525
    c1       (chain, draw) float32 1.6018301 10.666189 ... -6.6592984 -7.319274
Attributes:
    created_at:                 2020-02-28T13:03:09.683670
    arviz_version:              0.6.1
    inference_library:          pyro
    inference_library_version:  1.2.1

In [8]:
pyro_data.sample_stats

<xarray.Dataset>
Dimensions:    (chain: 3, draw: 50)
Coordinates:
  * chain      (chain) int64 0 1 2
  * draw       (draw) int64 0 1 2 3 4 5 6 7 8 9 ... 41 42 43 44 45 46 47 48 49
Data variables:
    diverging  (chain, draw) bool False False False True ... False False False
Attributes:
    created_at:                 2020-02-28T13:03:09.565887
    arviz_version:              0.6.1
    inference_library:          pyro
    inference_library_version:  1.2.1

In [9]:
pyro_data.log_likelihood

<xarray.Dataset>
Dimensions:         (chain: 3, developer: 5, draw: 50)
Coordinates:
  * chain           (chain) int64 0 1 2
  * draw            (draw) int64 0 1 2 3 4 5 6 7 8 ... 42 43 44 45 46 47 48 49
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (chain, draw, developer) float64 -13.09 -16.7 ... -9.237
    github_commits  (chain, draw, developer) float64 -3.567 -3.972 ... -2.433
Attributes:
    created_at:                 2020-02-28T13:03:09.588714
    arviz_version:              0.6.1
    inference_library:          pyro
    inference_library_version:  1.2.1

In [10]:
pyro_data.posterior_predictive

<xarray.Dataset>
Dimensions:         (chain: 3, developer: 5, draw: 50)
Coordinates:
  * chain           (chain) int64 0 1 2
  * draw            (draw) int64 0 1 2 3 4 5 6 7 8 ... 42 43 44 45 46 47 48 49
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (chain, draw, developer) float64 8.33e+03 ... 2.965e+04
    github_commits  (chain, draw, developer) float64 38.23 55.61 ... 67.03 91.18
Attributes:
    created_at:                 2020-02-28T13:03:09.608448
    arviz_version:              0.6.1
    inference_library:          pyro
    inference_library_version:  1.2.1

In [11]:
pyro_data.observed_data

<xarray.Dataset>
Dimensions:         (developer: 5)
Coordinates:
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (developer) int64 7500 10100 18600 25200 27500
    github_commits  (developer) int64 25 32 49 66 96
Attributes:
    created_at:                 2020-02-28T13:03:09.725104
    arviz_version:              0.6.1
    inference_library:          pyro
    inference_library_version:  1.2.1

In [12]:
pyro_data.prior_predictive

<xarray.Dataset>
Dimensions:         (chain: 1, developer: 5, draw: 50)
Coordinates:
  * chain           (chain) int64 0
  * draw            (draw) int64 0 1 2 3 4 5 6 7 8 ... 42 43 44 45 46 47 48 49
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (chain, draw, developer) float64 118.3 480.3 ... -609.7
    github_commits  (chain, draw, developer) float64 25.17 27.03 ... -149.5
Attributes:
    created_at:                 2020-02-28T13:03:09.703735
    arviz_version:              0.6.1
    inference_library:          pyro
    inference_library_version:  1.2.1

In [13]:
pyro_data.predictions

<xarray.Dataset>
Dimensions:         (chain: 3, developer_pred: 2, draw: 50)
Coordinates:
  * chain           (chain) int64 0 1 2
  * draw            (draw) int64 0 1 2 3 4 5 6 7 8 ... 42 43 44 45 46 47 48 49
  * developer_pred  (developer_pred) <U7 'Francis' 'Gerard'
Data variables:
    slack_comments  (chain, draw, developer_pred) float32 -4666.0923 ... 7561.4326
    github_commits  (chain, draw, developer_pred) float32 24.212748 ... 26.666
Attributes:
    created_at:                 2020-02-28T13:03:09.627837
    arviz_version:              0.6.1
    inference_library:          pyro
    inference_library_version:  1.2.1

In [14]:
pyro_data.constant_data

<xarray.Dataset>
Dimensions:            (developer: 5)
Coordinates:
  * developer          (developer) object 'Alice' 'Bob' ... 'Danielle' 'Erika'
Data variables:
    time_since_joined  (developer) float64 4.5 6.0 7.0 12.0 18.0
Attributes:
    created_at:                 2020-02-28T13:03:09.646651
    arviz_version:              0.6.1
    inference_library:          pyro
    inference_library_version:  1.2.1

In [15]:
pyro_data.predictions_constant_data

<xarray.Dataset>
Dimensions:            (developer_pred: 2)
Coordinates:
  * developer_pred     (developer_pred) <U7 'Francis' 'Gerard'
Data variables:
    time_since_joined  (developer_pred) float32 3.6 5.1
Attributes:
    created_at:                 2020-02-28T13:03:09.664658
    arviz_version:              0.6.1
    inference_library:          pyro
    inference_library_version:  1.2.1

In [9]:
newdata = az.from_pyro(predictions=predictions, pred_dims=pred_dims, coords=coords, num_chains=3)
newdata

Inference data with groups:
	> predictions

In [10]:
newdata.predictions

<xarray.Dataset>
Dimensions:         (chain: 3, developer_pred: 2, draw: 50)
Coordinates:
  * chain           (chain) int64 0 1 2
  * draw            (draw) int64 0 1 2 3 4 5 6 7 8 ... 42 43 44 45 46 47 48 49
  * developer_pred  (developer_pred) <U7 'Francis' 'Gerard'
Data variables:
    slack_comments  (chain, draw, developer_pred) float32 -4666.0923 ... 7561.4326
    github_commits  (chain, draw, developer_pred) float32 24.212748 ... 26.666
Attributes:
    created_at:                 2020-02-27T19:36:42.714726
    arviz_version:              0.6.1
    inference_library:          pyro
    inference_library_version:  1.2.1